In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import nltk
import spacy
from spacy import displacy
from spacy.tokens import DocBin
from tqdm import tqdm

In [3]:
data = pd.read_json('labels_new.json')
data.head()

,documents
0,"{'annotations': [{'end': 13, 'start': 3, 'tag_..."
1,"{'annotations': [{'end': 18, 'start': 3, 'tag_..."
2,"{'annotations': [{'end': 16, 'start': 3, 'tag_..."
3,"{'annotations': [{'end': 22, 'start': 3, 'tag_..."
4,"{'annotations': [{'end': 36, 'start': 13, 'tag..."


In [4]:
training_data = []

for document in data['documents']:
    text = document['content']
    
    entities = document['annotations']
    if len(text) < 50:
        print('missing contents')
        continue
    else:
        training_data.append({'text':text,'entities':entities})

missing contents
missing contents
missing contents
missing contents
missing contents
missing contents
missing contents
missing contents
missing contents
missing contents
missing contents
missing contents
missing contents
missing contents
missing contents
missing contents
missing contents
missing contents


In [5]:
nlp = spacy.blank("en") 
doc_bin = DocBin()

In [19]:
for d in tqdm(training_data):
    text = d['text']
    if len(text) < 50:
        continue
    else:
        labels = d['entities']
        doc = nlp.make_doc(text)
        entities = []

        for l in labels:
            start = l['start']
            if text[start] == ' ':
                start +=1
            end = l['end']
            if text[end] == ' ':
                end -=1

            span = doc.char_span(start_idx=start, end_idx=end,label=l['tag_name'])
            if span == None:
                span = doc.char_span(start_idx=start, end_idx=end,label='skill-unknown')
            entities.append(span)
        doc.set_ents(entities)
        doc_bin.add(doc)

doc_bin.to_disk("train.spacy")

  0%|          | 0/185 [00:00<?, ?it/s]


AttributeError: 'NoneType' object has no attribute 'label'

In [8]:
! python -m spacy init config config.cfg --lang en --pipeline ner --optimize efficiency


✘ The provided output file already exists. To force overwriting the
config file, set the --force or -F flag.



In [15]:
! python -m spacy train config.cfg --output ./ --paths.train ./train.spacy --paths.dev ./train.spacy

ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00    226.14    0.24    0.13    1.37    0.00
  1     200       1149.10  12544.04   37.47   52.19   29.23    0.37
  2     400        708.77   3834.57   63.52   72.21   56.70    0.64
^C


In [10]:
nlp_trained = spacy.load("model-best")

In [12]:
doc = nlp_trained(
"""Objective

Enthusiastic first-year Biochemistry undergraduate at the University of Wisconsin–Madison seeking a summer internship opportunity to apply laboratory knowledge, develop research skills, and contribute meaningfully to ongoing scientific work.
Education

University of Wisconsin–Madison
Bachelor of Science in Biochemistry
Expected Graduation: May 2028
GPA: 3.89/4.00
Relevant Coursework: General Chemistry I & II, Molecular Biology, Intro to Scientific Computing, Organic Chemistry (in progress)
Experience

Science Outreach Intern
Madison Science Museum — Madison, WI
June 2023 – August 2023

    Led weekly interactive science workshops for middle school students, focusing on genetics and microbiology.

    Assisted with curating and updating biology exhibits and lab experiment stations.

    Collaborated with a team of 5 interns to develop educational materials under supervision of the Outreach Coordinator.

    Gained experience in public speaking, scientific communication, and lesson planning.

Volunteer Lab Assistant
BioGenTech Laboratories — Milwaukee, WI
Summer 2022 (4 weeks shadowing experience)

    Observed protein purification techniques, DNA extraction protocols, and chromatography methods.

    Maintained sterile lab environments and helped prepare buffer solutions and culture media.

    Participated in weekly seminars on CRISPR, enzyme kinetics, and cell signaling pathways.

Skills

Technical Skills:

    Basic lab techniques: pipetting, spectrophotometry, pH measurement, gel electrophoresis

    Software: Microsoft Excel, R (beginner), Python (beginner), GraphPad Prism

    Report writing and scientific documentation

    Familiar with laboratory safety protocols and SDS guidelines

Soft Skills:

    Strong verbal and written communication

    Detail-oriented and highly organized

    Curious and proactive learner

    Comfortable working in teams and independently

    Passionate about science education and community outreach

Extracurricular Activities

Biochemistry Undergraduate Society
Member, Fall 2024 – Present

    Attend monthly guest speaker events and workshops on research careers.

STEM Ambassadors Program
High School Chapter President, 2022–2023

    Organized peer tutoring in biology and chemistry for over 40 students.

References

Available upon request."""
)



In [14]:
from IPython.display import HTML, display
html = displacy.render(doc, style="ent", jupyter=False)
display(HTML(f'<span class="tex2jax_ignore">{html}</span>'))